In [1]:
from indra.preassembler.grounding_mapper.gilda import ground_statements
from indra.preassembler.grounding_mapper.mapper import GroundingMapper
from indra.statements.statements import stmts_from_json
from indra.statements.statements import stmts_to_json
from indra_db.client.principal import raw_statements
from indra.tools import assemble_corpus as ac
from collections import defaultdict
#from indra_db.util import get_db
from datetime import datetime
import pickle as pk
import pandas as pd
import gilda
import time
#db = get_db('primary')

In [ ]:
#with open('./id_list.txt', 'r') as fh:
#    ids = [i.strip() for i in fh]

In [2]:
'''
tcid = ids
stmts = raw_statements.get_raw_stmt_jsons(clauses=[db.TextContent.id.in_(tcid)], db=db)
stmts_pkl = open('./raw_stmts_json.pkl', 'wb')
pk.dump(stmts, stmts_pkl)
stmts_json = list(stmts.values())
stmts_list = stmts_from_json(stmts_json)
outfh = open('./raw_stmts_list.pkl', 'wb')
pk.dump(stmts_list, outfh)
'''
infh = open('./raw_stmts_list.pkl', 'rb')
stmts_list = pk.load(infh)
start_time = time.time()
grounded_stmts = ac.map_grounding(stmts_list)
end_time = time.time()
round((end_time - start_time)/60, 2)
'''
INFO: [2021-10-06 11:27:28] indra.tools.assemble_corpus - Mapping grounding on 1333769 statements...
INFO: [2021-10-06 11:35:01] indra.preassembler.grounding_mapper.mapper - 44784 statements filtered out
7.56
'''

In [15]:
print(grounded_stmts[0:10][3].agent_list()[0],
      grounded_stmts[0:10][3].agent_list()[0].get_grounding(), 
      grounded_stmts[0:10][3].agent_list()[0].db_refs['TEXT']
     )

quercetin() ('CHEBI', 'CHEBI:16243') Quercetin


In [ ]:
def _check_attribute(ag):
    # Check for db refs
    if 'db_refs' in dir(ag):
        if 'TEXT' in ag.db_refs.keys():
            db_refs = True
        else:
            db_refs = False
    else:
        db_refs = False
    
    # Check for get_groundings
    if 'get_grounding' in dir(ag):
        db_grounding = True
    else:
        db_grounding = False
        
    return db_refs, db_grounding

df = []
for stmt in grounded_stmts:
    for ag in stmt.agent_list():
        
        # check if attributes exists
        db_refs, ag_grounding = _check_attribute(ag)
        
        if db_refs == True:
            agent_refs = ag.db_refs['TEXT']
        elif db_refs == False:
            agent_refs = 'NA'
        
        if ag_grounding == True:
            agent_grounding = ag.get_grounding()
            if agent_grounding == (None, None):
                if db_refs == True:
                    if 'TEXT' and 'TEXT_NORM' in ag.db_refs.keys():
                        agent_grounding = [ag.db_refs['TEXT'], ag.db_refs['TEXT_NORM']]
                        #print(ag.db_refs['TEXT'],ag.db_refs['TEXT_NORM'], end='\n\n') 
        elif ag_grounding == False:
            agent_grounding = ['NA', 'NA']
            

        df.append(
        {
            'Text': agent_refs,
            'db_ns': agent_grounding[0],
            'db_id': agent_grounding[1]

        })
grounded_df = pd.DataFrame(df)
grounded_df.to_csv('./agent_groundings.csv', sep=',')

In [47]:
'''l1 = [2,4,3]
l2 = [5,6,4]

def addTwoNumbers(l1, l2):
    n1 = int(''.join([str(i) for i in l1][::-1]))
    n2 = int(''.join([str(i) for i in l2][::-1]))
    ans = [i for i in str(n1+n2)][::-1]
    return ans
    
    
    
addTwoNumbers(l1, l2)'''

"l1 = [2,4,3]\nl2 = [5,6,4]\n\ndef addTwoNumbers(l1, l2):\n    n1 = int(''.join([str(i) for i in l1][::-1]))\n    n2 = int(''.join([str(i) for i in l2][::-1]))\n    ans = [i for i in str(n1+n2)][::-1]\n    return ans\n    \n    \n    \naddTwoNumbers(l1, l2)"